In [2]:
!pip install folium panda

  Preparing metadata (setup.py) ... done
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7239 sha256=5d872319e887d21da4028b977f49289bbd91f847650bd09333a0b8e87436d1ec
  Stored in directory: /root/.cache/pip/wheels/98/41/5b/6ca54e0b6a35e1b7248c12f56fcb753dfb7717fefaa0fb45f5
Successfully built panda


In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [8]:


# Load directly from URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
spacex_df = pd.read_csv(url)

# Create the map (use the previous code)
start_coords = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
spacex_map = folium.Map(location=start_coords, zoom_start=5)

for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(spacex_map)

spacex_map

In [11]:
import requests
import pandas as pd
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_df = pd.read_csv(io.StringIO(response.text))

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [12]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [13]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [14]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [16]:
import folium
import pandas as pd

# Center map on USA
usa_center = [39.8283, -98.5795]
site_map = folium.Map(location=usa_center, zoom_start=4)

# Create a dictionary with the 4 main SpaceX launch sites
launch_sites = {
    'CCAFS LC-40': {'lat': 28.562302, 'lon': -80.577356, 'color': 'red'},
    'CCAFS SLC-40': {'lat': 28.561857, 'lon': -80.577366, 'color': 'blue'},
    'KSC LC-39A': {'lat': 28.573255, 'lon': -80.646895, 'color': 'green'},
    'VAFB SLC-4E': {'lat': 34.632093, 'lon': -120.610829, 'color': 'purple'}
}

# Add markers for each launch site
for site_name, coords in launch_sites.items():
    # Add circle
    folium.Circle(
        location=[coords['lat'], coords['lon']],
        radius=5000,
        popup=f"<b>{site_name}</b>",
        color=coords['color'],
        fill=True,
        fill_opacity=0.3
    ).add_to(site_map)

    # Add marker with icon
    folium.Marker(
        location=[coords['lat'], coords['lon']],
        popup=f"""
        <div style='font-family: Arial;'>
        <h4>{site_name}</h4>
        <b>Location:</b> {coords['lat']:.4f}°, {coords['lon']:.4f}°<br>
        <b>State:</b> {'Florida' if 'CCAFS' in site_name or 'KSC' in site_name else 'California'}
        </div>
        """,
        tooltip=f"Click for {site_name} details",
        icon=folium.Icon(color=coords['color'], icon='rocket', prefix='fa')
    ).add_to(site_map)

# Add title
title_html = '''
             <h3 align="center" style="font-size:16px">
             <b>SpaceX Launch Sites in USA</b></h3>
             '''
site_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
site_map

In [17]:
from folium.plugins import MarkerCluster

# Create map with marker clusters
cluster_map = folium.Map(location=usa_center, zoom_start=4)
marker_cluster = MarkerCluster().add_to(cluster_map)

for index, row in spacex_df.iterrows():
    if 'class' in spacex_df.columns:
        is_success = row['class'] == 1
        color = 'green' if is_success else 'red'

        folium.Marker(
            location=[row['Lat'], row['Long']],
            popup=f"{row['Launch Site']}: {'Success' if is_success else 'Failure'}",
            icon=folium.Icon(color=color, icon='rocket', prefix='fa')
        ).add_to(marker_cluster)

cluster_map

In [18]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [19]:
marker_cluster = MarkerCluster()

In [20]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [21]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='circle'),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

In [22]:
# Task 3: Calculate distances between launch sites and nearby landmarks

# Function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points using Haversine formula
    Returns distance in kilometers
    """
    from math import radians, sin, cos, sqrt, atan2

    # Convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    # Earth radius in kilometers
    R = 6371.0
    distance = R * c

    return distance

# Define proximity points for each launch site (example coordinates)
proximity_points = {
    'CCAFS SLC-40': {
        'Railway': (28.562, -80.600),  # Approximate coordinates
        'Highway': (28.563, -80.590),
        'Coastline': (28.561, -80.567),
        'City': (28.509, -80.605)  # Cocoa Beach
    },
    'KSC LC-39A': {
        'Railway': (28.580, -80.650),
        'Highway': (28.575, -80.640),
        'Coastline': (28.573, -80.647),
        'City': (28.509, -80.605)  # Cocoa Beach
    },
    'VAFB SLC-4E': {
        'Railway': (34.630, -120.620),
        'Highway': (34.635, -120.615),
        'Coastline': (34.632, -120.611),
        'City': (34.639, -120.458)  # Lompoc
    }
}

# Calculate distances for each launch site
distance_results = []

for site_name, proximities in proximity_points.items():
    # Get launch site coordinates from dataframe
    site_coords = spacex_df[spacex_df['Launch Site'] == site_name][['Lat', 'Long']].iloc[0]
    site_lat, site_lon = site_coords['Lat'], site_coords['Long']

    for proximity_name, (prox_lat, prox_lon) in proximities.items():
        distance_km = calculate_distance(site_lat, site_lon, prox_lat, prox_lon)
        distance_miles = distance_km * 0.621371  # Convert to miles

        distance_results.append({
            'Launch Site': site_name,
            'Proximity': proximity_name,
            'Distance_km': round(distance_km, 2),
            'Distance_miles': round(distance_miles, 2)
        })

# Create DataFrame with results
distances_df = pd.DataFrame(distance_results)
print(distances_df)

# Visualize distances on map
distance_map = folium.Map(location=[28.5, -80.6], zoom_start=10)

# Add launch sites
for site_name in proximity_points.keys():
    site_coords = spacex_df[spacex_df['Launch Site'] == site_name][['Lat', 'Long']].iloc[0]

    # Add launch site marker
    folium.Marker(
        location=[site_coords['Lat'], site_coords['Long']],
        popup=f"<b>{site_name}</b>",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(distance_map)

    # Add lines to proximity points
    for proximity_name, (prox_lat, prox_lon) in proximity_points[site_name].items():
        # Add proximity point marker
        folium.Marker(
            location=[prox_lat, prox_lon],
            popup=f"<b>{proximity_name}</b><br>Distance to {site_name}: {distances_df[(distances_df['Launch Site']==site_name) & (distances_df['Proximity']==proximity_name)]['Distance_km'].values[0]} km",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(distance_map)

        # Add line connecting them
        folium.PolyLine(
            locations=[[site_coords['Lat'], site_coords['Long']], [prox_lat, prox_lon]],
            color='gray',
            weight=2,
            opacity=0.5,
            popup=f"Distance: {distances_df[(distances_df['Launch Site']==site_name) & (distances_df['Proximity']==proximity_name)]['Distance_km'].values[0]} km"
        ).add_to(distance_map)

# Display map
distance_map

     Launch Site  Proximity  Distance_km  Distance_miles
0   CCAFS SLC-40    Railway         2.27            1.41
1   CCAFS SLC-40    Highway         1.29            0.80
2   CCAFS SLC-40  Coastline         0.99            0.61
3   CCAFS SLC-40       City         6.63            4.12
4     KSC LC-39A    Railway         0.81            0.50
5     KSC LC-39A    Highway         0.70            0.44
6     KSC LC-39A  Coastline         0.03            0.02
7     KSC LC-39A       City         8.23            5.12
8    VAFB SLC-4E    Railway         0.90            0.56
9    VAFB SLC-4E    Highway         0.46            0.28
10   VAFB SLC-4E  Coastline         0.10            0.06
11   VAFB SLC-4E       City        13.99            8.69


In [23]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [24]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [26]:
# Interactive version - find closest coastline by sampling points

# Sample coastline points (you would get these from actual coastline data)
coastline_points = [
    (28.56367, -80.56799),
    (28.56200, -80.56850),
    (28.56100, -80.56900),
    (28.56000, -80.56950)
]

# Find closest coastline point
def find_closest_coastline(launch_lat, launch_lon, coastline_points):
    min_distance = float('inf')
    closest_point = None

    for coast_lat, coast_lon in coastline_points:
        distance = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)
        if distance < min_distance:
            min_distance = distance
            closest_point = (coast_lat, coast_lon, distance)

    return closest_point

# Get launch site coordinates (example for first launch site in dataframe)
first_site = spacex_df.iloc[0]
launch_site_lat, launch_site_lon = first_site['Lat'], first_site['Long']

# Find closest coastline
closest_coast = find_closest_coastline(launch_site_lat, launch_site_lon, coastline_points)
coastline_lat, coastline_lon, distance = closest_coast

print(f"Launch Site: ({launch_site_lat}, {launch_site_lon})")
print(f"Closest Coastline: ({coastline_lat}, {coastline_lon})")
print(f"Distance: {distance:.2f} km")

# Create the map
map_with_coastline = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Add launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(map_with_coastline)

# Add coastline marker with distance label
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(200, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 14pt; color: #2980b9; background: white; padding: 5px; border-radius: 3px; border: 1px solid gray;"><b>Coastline: {distance:.2f} km</b></div>'
    )
)
distance_marker.add_to(map_with_coastline)

# Add connecting line
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='#3498db',
    weight=3,
    dash_array='5, 5',
    popup=f'Distance to coastline: {distance:.2f} km'
).add_to(map_with_coastline)

# Add all coastline points for reference
for coast_lat, coast_lon in coastline_points:
    folium.CircleMarker(
        [coast_lat, coast_lon],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        popup=f'Coastline point: ({coast_lat}, {coast_lon})'
    ).add_to(map_with_coastline)

# Add MousePosition plugin
MousePosition().add_to(map_with_coastline)

map_with_coastline

Launch Site: (28.56230197, -80.57735648)
Closest Coastline: (28.56, -80.5695)
Distance: 0.81 km


In [28]:
# Create coordinates for the PolyLine
coordinates = [
    [launch_site_lat, launch_site_lon],  # Launch site
    [coastline_lat, coastline_lon]       # Coastline point
]

# Create PolyLine object
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,           # Line thickness
    color='blue',       # Line color
    opacity=0.8,        # Line transparency
    popup=f'Distance: {distance_coastline:.2f} km'  # Popup text
)

# Add PolyLine to the map
site_map.add_child(lines)

In [29]:
# Coordinates for different proximities (example - adjust with actual coordinates)
proximities = {
    'Closest City': {
        'coords': (28.509, -80.605),  # Cocoa Beach for Florida sites
        'color': 'blue'
    },
    'Nearest Railway': {
        'coords': (28.562, -80.600),
        'color': 'green'
    },
    'Nearest Highway': {
        'coords': (28.563, -80.590),
        'color': 'orange'
    },
    'Coastline': {
        'coords': (coastline_lat, coastline_lon),  # Use your coastline coordinates
        'color': 'purple'
    }
}

# Launch site coordinates (example)
launch_site_coords = (launch_site_lat, launch_site_lon)

# Create map centered on launch site
proximity_map = folium.Map(location=launch_site_coords, zoom_start=13)

# Add launch site marker
folium.Marker(
    location=launch_site_coords,
    popup='<b>Launch Site</b>',
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(proximity_map)

# Add proximity markers and lines
for name, info in proximities.items():
    proximity_coords = info['coords']
    color = info['color']

    # Calculate distance
    distance_km = calculate_distance(
        launch_site_coords[0], launch_site_coords[1],
        proximity_coords[0], proximity_coords[1]
    )

    # Create proximity marker with distance
    folium.Marker(
        location=proximity_coords,
        popup=f'<b>{name}</b><br>Distance: {distance_km:.2f} km<br>Coordinates: {proximity_coords[0]:.5f}, {proximity_coords[1]:.5f}',
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(proximity_map)

    # Draw line between launch site and proximity
    folium.PolyLine(
        locations=[launch_site_coords, proximity_coords],
        color=color,
        weight=2,
        opacity=0.7,
        popup=f'Distance to {name}: {distance_km:.2f} km'
    ).add_to(proximity_map)

    # Add distance label at midpoint
    midpoint = [
        (launch_site_coords[0] + proximity_coords[0]) / 2,
        (launch_site_coords[1] + proximity_coords[1]) / 2
    ]

    folium.Marker(
        location=midpoint,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 10pt; color: {color}; background: white; padding: 2px; border: 1px solid {color}; border-radius: 3px;"><b>{distance_km:.1f} km</b></div>'
        )
    ).add_to(proximity_map)

# Add legend
legend_html = '''
<div style="position: fixed; bottom: 50px; right: 50px; z-index: 1000; background-color: white; padding: 10px; border: 1px solid grey; border-radius: 5px;">
    <b>Proximity Legend</b><br>
    <i class="fa fa-circle" style="color:red"></i> Launch Site<br>
    <i class="fa fa-circle" style="color:blue"></i> City<br>
    <i class="fa fa-circle" style="color:green"></i> Railway<br>
    <i class="fa fa-circle" style="color:orange"></i> Highway<br>
    <i class="fa fa-circle" style="color:purple"></i> Coastline
</div>
'''
proximity_map.get_root().html.add_child(folium.Element(legend_html))

# Display map
proximity_map